In [1]:
import pandas as pd
import os

pd.options.mode.chained_assignment = None

# Read all json files in the output folder
results = {}
for file in os.listdir("output"):
    results[file[:-5]]=pd.read_json(os.path.join("output",file), orient="records")
results.keys()

dict_keys(['iclr_adult_antag_ds_rl_di', 'iclr_adult_antag_ds_rl_dp', 'iclr_adult_antag_ds_rl_eodds', 'iclr_adult_antag_ds_rl_eopp', 'iclr_adult_antag_ds_rl_pe', 'iclr_adult_fixed_ds_rl_di', 'iclr_adult_fixed_ds_rl_dp', 'iclr_adult_fixed_ds_rl_eodds', 'iclr_adult_fixed_ds_rl_eopp', 'iclr_adult_fixed_ds_rl_pe', 'iclr_brazil_antag_ds_rl_di', 'iclr_brazil_antag_ds_rl_dp', 'iclr_brazil_antag_ds_rl_eodds', 'iclr_brazil_antag_ds_rl_eopp', 'iclr_brazil_antag_ds_rl_pe', 'iclr_brazil_fixed_ds_rl_di', 'iclr_brazil_fixed_ds_rl_dp', 'iclr_brazil_fixed_ds_rl_eodds', 'iclr_brazil_fixed_ds_rl_eopp', 'iclr_brazil_fixed_ds_rl_pe'])

In [2]:
fc_map = {
    "di": "Disparate Impact",
    "dp": "Demographic Parity",
    "eodds": "Equalized Odds",
    "eopp": "Equal Opportunity",
    "pe": "Predictive Equality"
    }

model_map = {
    'SC'              : 'SC',
    'QSC'             : 'Quasi-SC',
    'QSRC'            : 'Shifty',
    'FairlearnSVC'    : 'Fairlearn',
    'FairConst'       : 'FairConst',
    'FairRobust'      : 'RFLearn'
}

def aggragate_table(df):
    df = df.loc[df["name"] != "SRC"]
    df["name"]=df["name"].map(model_map)
    df_large = df.loc[df["n_train"] == 60000][["name", "original_nsf", "antagonist_acc_mean", "antagonist_failed_mean"]].reset_index(drop=True)
    df_large["Delta Acc"] = df_large["antagonist_acc_mean"].reset_index(drop=True) - df.loc[df["n_train"] == 10000]["antagonist_acc_mean"].reset_index(drop=True)
    df_large.set_index("name", inplace=True)
    df_large.index.name = None
    return df_large

In [3]:
for dataset in ["adult", "brazil"]:
    for fc in ["di", "dp", "eodds", "eopp", "pe"]:
        fixed = results[f"iclr_{dataset}_fixed_ds_rl_{fc}"]
        antag = results[f"iclr_{dataset}_antag_ds_rl_{fc}"]
        fixed = aggragate_table(fixed)
        antag = aggragate_table(antag)
        combined = pd.concat({"Fixed Bounds": fixed, "Unknown Bounds": antag}, axis=1)
        combined.rename(columns={"original_nsf": "NSF", "antagonist_acc_mean": "Acc", "antagonist_failed_mean": "FR"}, inplace=True)
        latex = combined.style.format(
            na_rep="-", precision=3
            ).highlight_max(
                props="font-weight:bold", subset=[("Fixed Bounds", "Acc"), ("Unknown Bounds", "Acc")]
                ).highlight_min(
                    props="font-weight:bold", subset=[("Fixed Bounds", "FR"), ("Unknown Bounds", "FR")]
                    ).to_latex(
                        label=f"{fc}_{dataset}", caption=f"{fc_map[fc]} - {dataset} dataset", hrules=True, convert_css=True, multicol_align="c") # header = ["NSF", "acc orig", "acc depl"]
        latex = latex.replace("\midrule", "\cmidrule(r){2-5} \cmidrule{6-9}")
        print(latex)
        print()

\begin{table}
\caption{Disparate Impact - adult dataset}
\label{di_adult}
\begin{tabular}{lrrrrrrrr}
\toprule
 & \multicolumn{4}{c}{Fixed Bounds} & \multicolumn{4}{c}{Unknown Bounds} \\
 & NSF & Acc & FR & Delta Acc & NSF & Acc & FR & Delta Acc \\
\cmidrule(r){2-5} \cmidrule{6-9}
FairConst & 0 & 0.781 & \bfseries 1.000 & -0.003 & 0 & 0.791 & \bfseries 1.000 & -0.030 \\
RFLearn & 0 & \bfseries 0.786 & \bfseries 1.000 & 0.000 & 0 & 0.826 & \bfseries 1.000 & -0.002 \\
Fairlearn & 0 & 0.782 & \bfseries 1.000 & 0.000 & 0 & \bfseries 0.841 & \bfseries 1.000 & 0.003 \\
Quasi-SC & 1 & - & - & - & 1 & - & - & - \\
Shifty & 1 & - & - & - & 1 & - & - & - \\
SC & 1 & - & - & - & 1 & - & - & - \\
\bottomrule
\end{tabular}
\end{table}


\begin{table}
\caption{Demographic Parity - adult dataset}
\label{dp_adult}
\begin{tabular}{lrrrrrrrr}
\toprule
 & \multicolumn{4}{c}{Fixed Bounds} & \multicolumn{4}{c}{Unknown Bounds} \\
 & NSF & Acc & FR & Delta Acc & NSF & Acc & FR & Delta Acc \\
\cmidrule(r){2-5}